In [1]:
pip install -qU pypdf langchain langchain_community langchain_experimental langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 33.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.9/341.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Create link to external models in BigQuery

Based off of this article

https://cloud.google.com/blog/products/data-analytics/how-to-use-rag-in-bigquery-to-bolster-llms

In [20]:
# import pandas as pd
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

# authenticate to Google Cloud

GOOGLE_PROJECT = 'gristmill5'
credentials = service_account.Credentials.from_service_account_file("creds/gristmill5-e521e2f08f35.json")
client = bigquery.Client(GOOGLE_PROJECT, credentials)

# create link to embedding model

sql = """
CREATE OR REPLACE MODEL `gristmill5.rag_test.gecko_embedding_model`
REMOTE WITH CONNECTION `projects/gristmill5/locations/us/connections/vertex_ai`
OPTIONS (ENDPOINT = 'textembedding-gecko');
"""

client.query(sql, project=GOOGLE_PROJECT).result()

# create link to LLM

sql = """
CREATE OR REPLACE MODEL `gristmill5.rag_test.gemini_llm_model`
REMOTE WITH CONNECTION `projects/gristmill5/locations/us/connections/vertex_ai`
OPTIONS (ENDPOINT = 'gemini-1.0-pro');
"""

client.query(sql, project=GOOGLE_PROJECT).result()

# create table function that accepts a user query, finds similar chunks, and passes those chunks to the LLM

sql = """
CREATE OR REPLACE TABLE FUNCTION rag_test.rag_query(querys STRING, route_type STRING, words INT64, doc_source ARRAY <STRING>, selected_distance FLOAT64) AS (
with q_embeddings as (
  SELECT
    text_embedding,
    content
  FROM
    ML.GENERATE_TEXT_EMBEDDING(
      MODEL `rag_test.gecko_embedding_model`,
      (
        SELECT
        CAST(querys AS STRING) AS content
      )
    )
),

a_embeddings as (
  select * 
  from `rag_test.embeddings` 
  where source in UNNEST(doc_source)
  and embedding_type = FORMAT('%s', route_type)
  and FORMAT('%s', route_type) = 'summary'
),

v_search as (
  SELECT *
  FROM
    VECTOR_SEARCH( 
      (
        select * 
        from `rag_test.embeddings` 
        where source in UNNEST(doc_source)
        -- and statistics is not null
        and embedding_type = FORMAT('%s', route_type)
        and FORMAT('%s', route_type) = 'details'
      ),
      'text_embedding',
      (select * from q_embeddings where 1=1),
      top_k => 5
    )
  WHERE distance < selected_distance
)

SELECT *
FROM 
  ML.GENERATE_TEXT(
    MODEL
      `rag_test.gemini_llm_model`, 
      (
        -- query for when an answer needs to contain specific details
        SELECT
          CONCAT(FORMAT('Answer this question in less than %d words:\\n\\n %s \\n\\n', words, querys), '\\n\\n by using these text chunks: \\n\\n', STRING_AGG(base.chunk, '\\n')) AS prompt, 
          -- CONCAT(FORMAT('Summarize these text chunks in less than %d words:\\n\\n', words), STRING_AGG(base.chunk, '\\n')) AS prompt, 
          ARRAY_AGG(
            STRUCT(
              base.id as id,
              base.chunk as chunk,
              -- base.statistics as statistics, 
              base.embedding_type,
              -- base.ml_embed_text_status as status,
              distance as distance
            )
          ) source_ids
        FROM v_search

        -- query for when answer needs to be a summary
        UNION ALL SELECT 
          CONCAT(FORMAT('Summarize this text in less than %d words:\\n\\n', words), SUBSTRING(chunk, 1, 32760)) AS prompt, 
          [
            STRUCT(
              id,
              chunk,
              -- statistics, 
              embedding_type,
              -- {} as status,
              0.1 as distance
            )
          ] source_ids
        FROM a_embeddings
      ),
      STRUCT(
        0.4 AS temperature,
        300 AS max_output_tokens,
        0.5 AS top_p,
        5 AS top_k,
        TRUE AS flatten_json_output
      )
  )
)
"""

client.query(sql, project=GOOGLE_PROJECT).result()



### Ask Hacker News - 2024

In [6]:
sql = """
INSERT INTO TABLE `rag_test.embeddings` as
SELECT 'Ask HN' as source, *
FROM ML.GENERATE_TEXT_EMBEDDING(
  MODEL `rag_test.gecko_embedding_model`, (
    SELECT cast(id AS STRING) id, concat(title, ': ', text) as content 
    FROM `bigquery-public-data.hacker_news.full` 
    where text is not null
    and type = 'story'
    and timestamp > '2024-01-01'
    )
  )
"""

client.query(sql, project=GOOGLE_PROJECT).result()

In [ ]:
import sqlite3
import pandas as pd

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', None)

conn = sqlite3.connect('/Users/anthonychamberas/chat.db')
# conn = sqlite3.connect('~/Library/Messages/chat.db')

messages = pd.read_sql_query('''
    select distinct h.id phone_number, chj.chat_id, m.ROWID message_id, m.text, m.attributedBody, HEX(m.attributedBody) hex_message, m.date, m.handle_id, datetime(m.date/1000000000 + strftime("%s", "2001-01-01") ,"unixepoch","localtime") as date_utc 
    from chat_handle_join chj 
    inner join chat_message_join cmj 
        on chj.chat_id = cmj.chat_id 
        -- and chj.handle_id in (7,8,9)
    inner join message m 
        on cmj.message_id = m.ROWID 
    inner join handle h 
        on chj.handle_id = h.ROWID 
''', conn)

mapping =  dict.fromkeys(range(32))

messages['hex_message'] = messages['hex_message'].apply(lambda x: bytes.fromhex(x))
messages['decoded'] = messages['hex_message'].str.decode("utf-8", "ignore")
messages['cleaned'] = messages['decoded'].str.translate(mapping)
messages['stripped'] = messages['cleaned'].str.extract(r'\+(.*)iI')

messages['reps'] = messages['stripped'].str.extract(r'(\dx\d*)')
messages['dots'] = messages['stripped'].str.extract(r'[….|…|..|…|..\s|.. ](\d*)')
messages['comma'] = messages['stripped'].str.extract(r'(\d*)[,]')

#messages[['date_utc', 'chat_id', 'handle_id','stripped', 'reps', 'dots']].to_csv('cleaned.csv')
messages[['date_utc', 'phone_number', 'chat_id', 'handle_id','stripped', 'reps', 'dots']]

In [15]:
import os
import pandas as pd
import streamlist as st
from langchain_openai.embeddings import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]

embed = OpenAIEmbeddings(model="text-embedding-3-large")

chunk = ['embed this sentence.', 'search for this sentence']
df = pd.DataFrame(chunk, columns=['chunk'])

vectors = embed.embed_documents(df['chunk'])
df['vectors'] = pd.Series(vectors).to_numpy()



from google.cloud import bigquery
from google.oauth2 import service_account

GOOGLE_PROJECT = 'gristmill5'
credentials = service_account.Credentials.from_service_account_file("creds/gristmill5-e521e2f08f35.json")
client = bigquery.Client(GOOGLE_PROJECT, credentials)

job_config = bigquery.LoadJobConfig(autodetect=True)
#table_id = bigquery.Table('table') 
#table_id = client.create_table(table, exists_ok=True)

job = client.load_table_from_dataframe(df,"gristmill5.rag_test.table_id",job_config=job_config).result()



In [168]:
from utils.connectors import *

sql = f"""
    select * 
    from `rag_test.embeddings` 
    where source in UNNEST(['Tableau Zen - Visual Analytics Maturity Assessment.docx'])
"""

data = bq_conn(sql)

query = 'How does it work?'
query = query.replace("'", "\\'")
embed = OpenAIEmbeddings(model="text-embedding-3-large")
vector = embed.embed_documents([query])

# Calculate cosine similarities between the query vector and the dataset
vectors = np.array(data['vectors'].to_list())
# similarities = cosine_similarity(vectors, vector)

# similarity_array = [s[0] for s in similarities]
# similarity_df = pd.DataFrame(similarity_array, columns=['similarity'])
similarities = pd.DataFrame([s[0] for s in cosine_similarity(vectors, vector)], columns=['similarity'])
df = pd.concat([data, similarities], axis=1)

n = 2
# top_n_idx = np.argsort(similarity_array)[-n:]
top_n_idx = np.argsort(df['similarity'])[-n:]
references = df[['source', 'page', 'chunk', 'similarity']].iloc[top_n_idx]

display(references)


,source,page,chunk,similarity
8,Tableau Zen - Visual Analytics Maturity Assess...,0,Why does it matter to me?,0.280168
4,Tableau Zen - Visual Analytics Maturity Assess...,0,How does it work?,0.999999
